In [ ]:
import HTSeq, os, importlib, pandas, re, collections, sys
from pathlib import Path

#os.chdir('/Users/dp/pma/dataAndScripts/clip/miseq/v2all/')
sys.path.append('/Users/dp/pma/')
import sameRiver
import sameRiver.peakFinder
import sameRiver.peakSeqFinder
import sameRiver.peakSignalFinder

importlib.reload(sameRiver.peakFinder)
importlib.reload(sameRiver.peakSeqFinder)
importlib.reload(sameRiver.peakSignalFinder)



In [ ]:
genomic_fasta_fname = '/opt/genomes/repeats_and_ensembl_release94_GRCh38/combined.fa'  # Old, repeats-first mapping
#genomic_fasta_fname = '/opt/genomes/gencode.v29/GRCh38.primary_assembly.genome.fa.gz'  # Newer, separate mapping.
genomic_fasta = dict( (s[1], s[0]) for s in HTSeq.FastaReader(genomic_fasta_fname, raw_iterator=True) )

In [ ]:
print(genomic_fasta_fname)

In [ ]:
importlib.reload(sameRiver.peakSeqFinder)

top_dir = '/Users/dp/pma/dataAndScripts/clip/old_mapping_meta/'
protein_coding = pandas.read_csv(f'{top_dir}/temp/enst_transcript_id_name_biotype_map.txt', sep='\t')
protein_coding = protein_coding.loc[[x=='protein_coding' for x in protein_coding.transcript_biotype],:]
protein_coding = set(protein_coding['gene_name'])

def write_fasta(prot, locs, pvals_at_peak):

    # This heights file is raw read counts, which we only use to sort by height.
    heights = pandas.read_excel(
        Path(top_dir, 'data', f'signal_at_{prot}_peak_locations.xlsx'),
        index_col=0)
    
    heights = heights[[x for x in heights.columns if (prot in x)]]

    print("Getting sequences for each peak...")

    cutoff = 2#0.2  # P value cutoff for signals at peak locations.
    
    if cutoff < 2:
        # Subset peak locations to those with P<cutoff.
        if prot not in pvals_at_peak:
            print(f"{prot} not in {Path(top_dir, 'tables', 'pvals_at_peaks_per_read.xlsx')}")
            return
        rnas_pass_filter = pvals_at_peak[pvals_at_peak[prot]<cutoff].copy()
        print(f'pass pvalue cutoff {cutoff} = {rnas_pass_filter.shape}')
        locs_past_filter = locs[[(x in rnas_pass_filter.index) for x in locs.index]]
    else:
        locs_past_filter = locs
        
    print('locations that passed p value filter:', locs_past_filter.shape)
    
    # Remove all ncRNA.
    locs_past_filter = locs_past_filter.loc[[x in protein_coding for x in locs_past_filter.index], :]
    
    pvals_per_read = pandas.read_excel(
        Path(top_dir, 'tables', 'pvals_per_read.xlsx'), index_col=0)    
    
    # P value cutoff for total signal in the given RNA.
    cutoff = 2#0.2
    
    if cutoff < 2:
        rnas_pass_filter = pvals_per_read[pvals_per_read[prot]<cutoff].copy()
        targets = [x.split('::')[0] for x in rnas_pass_filter.index]

        print('pass pvalue cutoff = ', rnas_pass_filter.shape)
        locs_past_filter = locs_past_filter[[(x in targets) for x in locs_past_filter.index]]
        print('locations that passed p value filter:', locs_past_filter.shape)

        heights = heights[[x in locs_past_filter.index for x in heights.index]]

    heights['sum'] = heights.sum(axis=1)
    to_height = dict(zip(heights.index, heights['sum']))
    locs_past_filter['height'] = [to_height.get(x, -100) for x in locs_past_filter.index]

    top_n = 10000
    min_height = 5#10
    
    if len(locs_past_filter.index) > top_n:
        locs_past_filter.sort_values(by='height', inplace=True, ascending=False)
        locs_past_filter = locs_past_filter.head(top_n).copy()
        
    if min_height:
        locs_past_filter = locs_past_filter.loc[[x>=min_height for x in locs_past_filter.height], :]
        
    print("Past height cutoff:", locs_past_filter.shape )

    if locs_past_filter.shape[0] == 0:
        return

    # Find and write fasta sequences.
    a = pkseq.get_seqs(locs_past_filter[[prot]])
    pkseq.write_seqs_for_motif_searches(outfolder=f'{top_dir}/data/seqs/', write_controls=True)
    
locs = pandas.read_excel(f'{top_dir}/data/peak_locations.xlsx', index_col=0)

#nb = sameRiver.statsForCountsNB.statsForCountsNB.load()

pkseq = sameRiver.peakSeqFinder.peakSeqFinder(fasta_dict=genomic_fasta)
pvals_at_peak = pandas.read_excel(
    Path(top_dir, 'tables', 'pvals_at_peaks_per_read.xlsx'),
    #'/Users/dfporter/pma/dataAndScripts/clip/miseq/v2all/data/pvals_for_signal_at_{}_peaks_per_read.xlsx'.format(prot),
    index_col=0)
#print("...... pvals_at_peak=", pvals_at_peak)
selected_proteins = [
    'CELF1',
    'PCBP1',
    'hnRNPD', 
    'Rbfox2',
    'Rbfox1', 
    'FBL',
    'hnRNPC', 
    'SF3B1',
    'A1CF',
    'KHDRBS2',
    'FUBP1',
#     'RARS2', 'RPL5',
    #'UBA2', 'ETS2',
#     'CRNKL1',
     #'CCIN',
]
#selected_proteins = ['SF3B1', 'Rbfox2', 'CELF1']
for prot in locs.columns:
    if prot not in selected_proteins:
        continue
    print('\n', prot)
    write_fasta(prot, locs, pvals_at_peak)
#rna_obj = pickle.load(open('./data/rna.data', 'rb'))
homer_command = """
# For some reason, homer1 works while homer2 doesn't.

export PATH=$PATH:/Users/dp/homer/bin/
"""
seq_dir = f'{top_dir}/data/seqs/'
as_one_cmd = []
outdir = './output/'
print(f'mkdir {outdir}')

for prot in locs.columns:
    if prot not in selected_proteins:
        continue
    cmd = "{fp} {d}/{g}.fa  fasta {o}/{g} -fasta {d}/randoms_for_{g}.fa".format(fp='./bin/findMotifs.pl', d=seq_dir, g=prot, o=outdir)
    cmd += ' -rna -len 3,4,5,6,7,8,9,12'  # -len 3,4,5,6,8,9,12
    print(cmd)
    as_one_cmd.append(cmd)
    
print(f'mkdir {outdir}/; rm {outdir}/*/*; rm {outdir}/*/*/*;' + '; '.join(as_one_cmd))

# For Rbfox2, what seems to work is a cutoff of 0.1,
# and this homer command ./bin/findMotifs.pl /Users/dfporter/pma/dataAndScripts/clip/miseq/v2all/data/seqs//Rbfox2.fa \
# fasta ./Rbfox2 -fasta /Users/dfporter/pma/dataAndScripts/clip/miseq/v2all/data/seqs//randoms_for_Rbfox2.fa  \
# -rna
# Adding -homer1 doesn't generally make a huge difference



In [ ]:
import os, pandas, importlib
import sameRiver
import sameRiver.negativeCounts
import sameRiver.scheme
import sameRiver.annCounts
import sameRiver.positiveCounts
importlib.reload(sameRiver.annCounts)
importlib.reload(sameRiver.negativeCounts)
os.chdir('/Users/dfporter/pma/dataAndScripts/clip/miseq/v2all/')
top_dir = './'

class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
negative_metadata = Namespace(
    top_dir = top_dir,
    scheme_file_with_random_proteins = top_dir + '/scheme.xlsx',
    ann_counts_file = top_dir + '/ann_counts.txt',
    bed_file_dir = top_dir + '/beds/',
    random_proteins = [
        'CAPNS2', #'CCIN',
        'CDK4', 'CHMP3',
        'DCTN6', #'EPB41L5',
        'ETS2', 'IDE',
        'ITPA', 'TPGS2', 'UBA2',
    ])

positive_metadata = Namespace(
    top_dir = top_dir,
    scheme_file = top_dir + '/scheme.xlsx',
    ann_counts_file = top_dir + '/ann_counts.txt',
    bed_file_dir = top_dir + '/beds/',
    positive_proteins = [
        'CAPNS2', 'CCIN', 'CDK4', 'CHMP3',
        'DCTN6', 'EPB41L5', 'ETS2', 'IDE',
        'ITPA', 'TPGS2', 'UBA2',
        'FBL', 'HCT116', 'hnRNPC',
        'SF3B1',
        'PCBP1', 'PCBP1:100P', 'PCBP1:100Q',
        'PCBP1:dKH', 'CELF1',
        'Rbfox1', 'Rbfox2', 'hnRNPD',
    ]
)
positives = sameRiver.positiveCounts.positiveCounts(positive_metadata)
positives.normalize_ann_counts(use_bed_dir='./beds/', load_total_read_numbers=True)
negatives = sameRiver.negativeCounts.negativeCounts(negative_metadata)
negatives.normalize_ann_counts(use_bed_dir='./beds/', load_total_read_numbers=True)
negatives.get_lowest_values_for_all_proteins()




In [ ]:
import sameRiver.statsForCountsNB
importlib.reload(sameRiver.statsForCountsNB)
importlib.reload(sameRiver.scheme)

scheme = sameRiver.scheme.scheme(scheme_fname='./scheme.xlsx')
a = time.time()
nb = sameRiver.statsForCountsNB.statsForCountsNB(negatives=negatives, positives=positives)
nb.calculate_pvalues(cutoff_for_consideration=1, test_mode=True)
b = time.time()
print('time {}m'.format((b - a)/60.))
nb.write_pvals_single_file(which='per_read', outfname='./tables/pvals_per_read.xlsx')

In [ ]:
fname = '/Users/dfporter/pma/dataAndScripts/clip/miseq/v2all/data/seqs/randoms_for_Rbfox1.fa'
lines = open(fname).readlines()
fa = ''.join([">{}\n{}".format(line.split('\t')[0], line.split('\t')[1]) for line in lines])
with open('/Users/dfporter/pma/dataAndScripts/clip/miseq/v2all/data/seqs/randoms_for_Rbfox1.fa', 'w') as f:
    f.write(fa)

In [ ]:

"""
Assemble outputs (made in write_fasta_for_homer.ipynb) into one file.
"""

import glob
from pathlib import Path

os.system('export PATH=$PATH:/Users/dp/homer/bin/')
print('export PATH=$PATH:/Users/dp/homer/bin/')
for fname in glob.glob(top_dir+'/data/seqs/*fa'):
    if 'random' in fname:
        continue

    gene = os.path.basename(fname).split('.fa')[0]
    random_fname = os.path.dirname(fname) + '/randoms_for_{0}.fa'.format(gene)
    homer = '/Users/dfporter/homer/bin/findMotifs_edit.pl'
    cmd = "{} {} fastq ./{} -fasta {} -rna -homer1".format(homer, fname, gene, random_fname)
    #print(cmd)
    #os.system(cmd)
    
class logoAssembly:
    
    def __init__(self, homer_dir='./'):
        self.homer_dir = homer_dir
    
    def find_all_svg_motif1_logos(self):
        print(f'{self.homer_dir}/*/motif1.logo.svg')
        svgs_motif1, svgs_motif2 = {}, {}
        for path in glob.glob(f'{self.homer_dir}/*/*/motif1.logo.svg'):
            with open(path) as f:
                svgs_motif1[os.path.dirname(path)] = f.readlines()
                
        for path in glob.glob(f'{self.homer_dir}/*/*/motif2.logo.svg'):
            print(path)
            with open(path) as f:
                svgs_motif2[os.path.dirname(path)] = f.readlines()
        #print(svgs)
        
        header = """<HTML><HEAD><TITLE>Homer de novo Motif Results</TITLE></HEAD>
<BODY>
<H1>Homer <i>de novo</i> Motif Results </H1>
"""
        print(os.getcwd())
        comb = open(self.homer_dir+'/all_logos.html', 'w')
        comb.write(header)
        for path, lines in svgs_motif1.items():
            
            comb.write('<H2>{}</H2>\n'.format(os.path.dirname(path)))
            comb.write(''.join(lines))
            if path in svgs_motif2:
                comb.write(''.join(svgs_motif2[path]))                
        comb.close()
        
logos = logoAssembly(
#    homer_dir='/Users/dp/homer/output/'
    homer_dir='/Users/dp/homer/top10Ktargets_5_min_height_no_Pvalue_cutoffs_20k_controlSeqs_mRNA_only_old_mapping_eg_repeats_first'
)
logos.find_all_svg_motif1_logos()